### Imports

In [1]:
import numpy as np 
import pandas as pd
import requests  
import pycountry 
from google.oauth2 import service_account
from googleapiclient.discovery import build
from functions import *
from conf.settings import *
import re
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Data Extraction and Casting

In [2]:
df_metrics_by_country = get_named_range_as_dataframe(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID, COUNTRY_NAMED_RANGE)
df_metrics_by_country = convert_column_names_to_standard_english(df_metrics_by_country)
df_metrics_by_country = clean_and_cast_columns(df_metrics_by_country)

### Analysis

In [3]:
df_metrics_by_country['campaign_goal'] = df_metrics_by_country['ad_set_name'].apply(create_column_campaign_goal)
df_metrics_by_country['classified_campaign_goal'] = df_metrics_by_country['campaign_goal'].apply(classify_campaign)
df_metrics_by_country['classified_result_indicator'] = df_metrics_by_country['result_indicator'].apply(lambda x: 'pixel_initiate_checkout' if 'fb_pixel_initiate_checkout' in x else  
                                                                                                         'pixel_add_to_cart' if 'fb_pixel_add_to_cart' in x else              
                                                                                                         'pixel_purchase' if 'fb_pixel_purchase' in x else                    
                                                                                                         'post_engagement' if 'post_engagement' in x else
                                                                                                         'offsite_conversion' if 'offsite_conversion' in x else                     
                                                                                                         'Outros')

In [4]:
print(df_metrics_by_country['classified_campaign_goal'].value_counts())
print(df_metrics_by_country['country'].value_counts())
print(df_metrics_by_country['result_indicator'].value_counts())

classified_campaign_goal
Conversão                   7392
Engajamento/Visualização    4051
Outros                       234
Name: count, dtype: int64
country
DE    630
BE    618
IT    610
NL    608
CH    597
FR    590
ES    575
GB    548
AT    539
US    310
AU    295
CA    262
LU    213
IE    193
DK    137
PT    125
NZ    118
NO    108
SE     72
FI     63
GR     55
IS     40
HK     38
BH     32
OM     32
AE     30
MC     30
QA     30
KW     29
SA     28
ID     24
MX     24
KR     24
MY     24
DZ     23
VE     23
TH     23
TZ     23
AO     23
NI     23
XK     23
TN     23
HU     23
IQ     23
LY     23
GH     22
AM     22
NG     22
NA     22
MK     22
RO     22
MD     22
AI     22
LK     22
KE     22
BA     22
GE     22
CO     22
IN     22
UG     22
SV     22
EG     22
HN     22
GT     22
PK     22
TR     22
SG     22
DO     22
SK     22
AL     22
MZ     22
JO     22
IL     22
EC     22
PS     22
BY     22
KH     22
BG     22
MU     22
NP     22
BD     22
KZ     22
RS     22
MA     22
SR

In [5]:
country_grouped = (df_metrics_by_country.groupby(["country", "classified_campaign_goal", "classified_result_indicator"]).agg({"results": "sum",                      
                                                                                                                  "result_rate": "mean",      
                                                                                                                  "reach": "sum",                     
                                                                                                                  "frequency": "mean",             
                                                                                                                  "cpc_link": "mean",                     
                                                                                                                  "cpc_general": "mean",                   
                                                                                                                  "cost_per_1000_people_reached": "mean",       
                                                                                                                  "ctr": "mean",                          
                                                                                                                  "add_to_cart": "sum",            
                                                                                                                  "cost_per_add_to_cart": "mean",           
                                                                                                                  "initiate_checkout": "sum",                    
                                                                                                                  "cost_per_initiate_checkout": "mean",        
                                                                                                                  "purchase": "sum",                 
                                                                                                                  "cost_per_purchase": "mean",             
                                                                                                                  "amount_spent": "sum",                       
                                                                                                                  "purchase_conversion_value": "sum",
                                                                                                                  "links_clicks": "sum",       
                                                                                                                  }).reset_index()
                                                                                                                  .rename(columns={'ad_set_name': 'qty_ad_sets'}))

country_grouped["roi"] = country_grouped["purchase_conversion_value"] / country_grouped["amount_spent"]
country_grouped["cart_to_checkout_conversion"] = np.where(country_grouped["add_to_cart"] > 0, 
                                                      country_grouped["initiate_checkout"] / country_grouped["add_to_cart"], 
                                                      0)
country_grouped["checkout_to_purchase_conversion"] = np.where(country_grouped["initiate_checkout"] > 0,  
                                                          country_grouped["purchase"] / country_grouped["initiate_checkout"], 
                                                          0)

### External Data Extraction 

#### Extracão via APIs do banco mundial e restcountries

In [6]:
country_grouped.columns

Index(['country', 'classified_campaign_goal', 'classified_result_indicator',
       'results', 'result_rate', 'reach', 'frequency', 'cpc_link',
       'cpc_general', 'cost_per_1000_people_reached', 'ctr', 'add_to_cart',
       'cost_per_add_to_cart', 'initiate_checkout',
       'cost_per_initiate_checkout', 'purchase', 'cost_per_purchase',
       'amount_spent', 'purchase_conversion_value', 'links_clicks', 'roi',
       'cart_to_checkout_conversion', 'checkout_to_purchase_conversion'],
      dtype='object')

In [7]:
names_mapping = get_country_full_names(list(country_grouped.country))  
country_grouped["full_country_name"] = country_grouped["country"].map(names_mapping)

In [8]:
valores = list(names_mapping.values())
population = get_population_by_country(valores)
country_grouped["country_population"] = country_grouped["full_country_name"].map(population)

Erro ao obter dados para None: 404 Client Error: Not Found for url: https://restcountries.com/v3.1/name/None?fullText=true
Erro ao obter dados para Brunei Darussalam: 404 Client Error: Not Found for url: https://restcountries.com/v3.1/name/Brunei%20Darussalam?fullText=true
Erro ao obter dados para Bolivia, Plurinational State of: 404 Client Error: Not Found for url: https://restcountries.com/v3.1/name/Bolivia,%20Plurinational%20State%20of?fullText=true
Erro ao obter dados para Congo, The Democratic Republic of the: 404 Client Error: Not Found for url: https://restcountries.com/v3.1/name/Congo,%20The%20Democratic%20Republic%20of%20the?fullText=true
Erro ao obter dados para Congo: 404 Client Error: Not Found for url: https://restcountries.com/v3.1/name/Congo?fullText=true
Erro ao obter dados para Côte d'Ivoire: 404 Client Error: Not Found for url: https://restcountries.com/v3.1/name/C%C3%B4te%20d'Ivoire?fullText=true
Erro ao obter dados para Cabo Verde: 404 Client Error: Not Found for ur

In [9]:
country_grouped['gdp_per_capita'] = country_grouped['country'].apply(get_gdp_per_capita)

In [10]:
print(country_grouped.columns)
country_grouped.shape

Index(['country', 'classified_campaign_goal', 'classified_result_indicator',
       'results', 'result_rate', 'reach', 'frequency', 'cpc_link',
       'cpc_general', 'cost_per_1000_people_reached', 'ctr', 'add_to_cart',
       'cost_per_add_to_cart', 'initiate_checkout',
       'cost_per_initiate_checkout', 'purchase', 'cost_per_purchase',
       'amount_spent', 'purchase_conversion_value', 'links_clicks', 'roi',
       'cart_to_checkout_conversion', 'checkout_to_purchase_conversion',
       'full_country_name', 'population_country', 'gdp_per_capita'],
      dtype='object')


(1015, 26)

### Save to sheets

In [11]:
column_order_english = [
    "country",
    "full_country_name",
    "population_country",
    "gdp_per_capita",
    "classified_campaign_goal",
    "classified_result_indicator",
    "amount_spent",
    "reach",
    "frequency",
    "cost_per_1000_people_reached",
    "links_clicks",
    "cpc_link",
    "cpc_general",
    "ctr",
    "results",
    "result_rate",
    "add_to_cart",
    "cost_per_add_to_cart",
    "initiate_checkout",
    "cost_per_initiate_checkout",
    "purchase",
    "cost_per_purchase",
    "purchase_conversion_value",
    "checkout_to_purchase_conversion",
    "cart_to_checkout_conversion",
    "roi"
]

In [12]:
save_df_to_named_range(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID, 'df_w_metrics_by_country', country_grouped[column_order_english])

{'spreadsheetId': '1HVlgRHzBjLS4yO9bOKrTeJwlitn9u40CU1rSFwH4VZo',
 'updatedRange': "'Analytics COUNTRY'!B2:AA1017",
 'updatedRows': 1016,
 'updatedColumns': 26,
 'updatedCells': 26416}

In [14]:
df_metrics_by_country.head()

,ad_set_name,country,result_rate,result_indicator,results,reach,frequency,links_clicks,cpc_link,cpc_general,cost_per_1000_people_reached,ctr,add_to_cart,cost_per_add_to_cart,initiate_checkout,cost_per_initiate_checkout,purchase,cost_per_purchase,amount_spent,purchase_conversion_value,campaign_goal,classified_campaign_goal,classified_result_indicator
0,"LC Engagement 05/02/2017 (13-65+, WW, Instagram)",GU,0.1596,actions:post_engagement,379,1533.0,1.55,6,0.01,0.00,0.02,0.0093,10,0.00,1,0.03,1.0,0.03,0.03,24.90,LC Engagement,Engajamento/Visualização,post_engagement
1,"LC Engagement 29/08/2016 (13-65+, WW, Hearts)",ES,0.0461,actions:post_engagement,191,4095.0,1.01,6,0.01,0.00,0.01,0.0533,3,0.01,1,0.03,1.0,0.03,0.03,24.90,LC Engagement,Engajamento/Visualização,post_engagement
2,"LC Engagement 29/08/2016 (13-65+, WW, Hearts)",SK,0.0787,actions:post_engagement,282,3071.0,1.17,34,0.00,0.00,0.01,0.0843,2,0.02,1,0.04,1.0,0.04,0.04,29.40,LC Engagement,Engajamento/Visualização,post_engagement
3,"LC Engagement 29/08/2016 (13-65+, WW, Hearts) - Instagram",CW,0.1394,actions:post_engagement,198,1023.0,1.39,15,0.00,0.00,0.06,0.0099,2,0.03,1,0.06,1.0,0.06,0.06,47.43,LC Engagement,Engajamento/Visualização,post_engagement
4,Viewed - 1 day - Rest of the World,AL,0.1471,actions:offsite_conversion,5,16.0,2.13,0,0.00,0.04,4.38,0.0588,1,0.07,1,0.07,1.0,0.07,0.07,21.90,Viewed,Engajamento/Visualização,Outros


### Metrics by Conversion and Engagement

In [40]:
def calcular_percentuais_por_objetivo(df, main_column, classified_campaign_goal='Conversão'):
    df_filtrado = df[df['classified_campaign_goal'] == classified_campaign_goal]  
    
    df_agrupado = df_filtrado.groupby([main_column, 'classified_campaign_goal'], as_index=False).agg(  
        results=('results', 'sum'),                     
        purchase=('purchase', 'sum'),                    
        purchase_conversion_value=('purchase_conversion_value', 'sum'),  
        amount_spent=('amount_spent', 'sum'),            
        count=('results', 'size')                        
    )
    
    df_agrupado = df_agrupado.sort_values(by='amount_spent', ascending=False)  
    
    
    if len(df_agrupado) > 12:  
        df_top12 = df_agrupado.iloc[:12].copy()
        df_outros = df_agrupado.iloc[12:].copy()
        outros_row = {  
            main_column: 'Outros',  
            'classified_campaign_goal': classified_campaign_goal,  
            'results': df_outros['results'].sum(),  
            'purchase': df_outros['purchase'].sum(),  
            'purchase_conversion_value': df_outros['purchase_conversion_value'].sum(),  
            'amount_spent': df_outros['amount_spent'].sum(), 
            'count': df_outros['count'].sum() 
        }
        
        df_outros_agg = pd.DataFrame([outros_row])  
        df_final = pd.concat([df_top12, df_outros_agg], ignore_index=True)  
    else:
        df_final = df_agrupado.copy()  
    
    
    total_results = df_final['results'].sum()  
    total_purchase = df_final['purchase'].sum()  
    total_pc_value = df_final['purchase_conversion_value'].sum()  
    total_spent = df_final['amount_spent'].sum() 
    total_count = df_final['count'].sum()
    
    df_final['pct_results'] = df_final['results'] / total_results if total_results != 0 else 0  
    df_final['pct_purchase'] = df_final['purchase'] / total_purchase if total_purchase != 0 else 0 
    df_final['pct_purchase_conversion_value'] = (df_final['purchase_conversion_value'] / total_pc_value) if total_pc_value != 0 else 0  
    df_final['pct_amount_spent'] = df_final['amount_spent'] / total_spent if total_spent != 0 else 0 
    df_final['pct_count'] = df_final['count'] / total_count if total_count != 0 else 0  
    
    for col in ['pct_results', 'pct_purchase', 'pct_purchase_conversion_value', 'pct_amount_spent', 'pct_count']:
        df_final[col] = (df_final[col]).round(2)
    
    
    return df_final 

In [44]:
df_metricas_by_country_conversao = calculate_percentages_by_goal(df_metrics_by_country, main_column='country')
df_metricas_by_country_conversao["full_country_name"] = df_metricas_by_country_conversao["country"].map(names_mapping)
df_metricas_by_country_conversao["country_population"] = df_metricas_by_country_conversao["full_country_name"].map(population)
df_metricas_by_country_conversao['gdp_per_capita'] = df_metricas_by_country_conversao['country'].apply(get_gdp_per_capita)
save_df_to_named_range(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID, 'df_analysis_by_country_conversion', df_metricas_by_country_conversao)
df_metricas_by_country_conversao

,country,classified_campaign_goal,results,purchase,purchase_conversion_value,amount_spent,count,pct_results,pct_purchase,pct_purchase_conversion_value,pct_amount_spent,pct_count,full_country_name,country_population,gdp_per_capita
0,DE,Conversão,9535,1102.0,34377.99,27081.42,577,0.12,0.13,0.13,0.14,0.08,Germany,83240525.0,54343.226508
1,IT,Conversão,8499,962.0,31314.60,25926.03,553,0.11,0.11,0.12,0.13,0.07,Italy,59554023.0,39003.316095
2,ES,Conversão,5437,656.0,21293.30,18499.65,522,0.07,0.08,0.08,0.09,0.07,Spain,47351567.0,33509.012798
3,NL,Conversão,7289,832.0,24555.13,17434.61,555,0.09,0.10,0.09,0.09,0.08,Netherlands,16655799.0,64572.005956
4,FR,Conversão,6781,774.0,24112.43,16623.40,538,0.09,0.09,0.09,0.08,0.07,France,67391582.0,44690.934540
5,US,Conversão,10094,871.0,25301.74,16573.12,269,0.13,0.10,0.09,0.08,0.04,United States,329484123.0,82769.412211
6,BE,Conversão,7308,800.0,27109.19,15395.16,558,0.09,0.09,0.10,0.08,0.08,Belgium,11555997.0,54700.909324
7,GB,Conversão,5696,742.0,24686.87,15144.09,509,0.07,0.09,0.09,0.08,0.07,United Kingdom,67215293.0,49463.855462
8,CH,Conversão,3618,472.0,17133.93,9491.34,539,0.05,0.06,0.06,0.05,0.07,Switzerland,8654622.0,99564.710026
9,CA,Conversão,2617,336.0,7430.27,8087.13,237,0.03,0.04,0.03,0.04,0.03,Canada,38005238.0,53431.185706


In [45]:
df_metricas_by_country_engajamento = calculate_percentages_by_goal(df_metrics_by_country, main_column='country', classified_campaign_goal='Engajamento/Visualização')
df_metricas_by_country_engajamento["full_country_name"] = df_metricas_by_country_engajamento["country"].map(names_mapping)
df_metricas_by_country_engajamento["country_population"] = df_metricas_by_country_engajamento["full_country_name"].map(population)
df_metricas_by_country_engajamento['gdp_per_capita'] = df_metricas_by_country_engajamento['country'].apply(get_gdp_per_capita)
save_df_to_named_range(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID, 'df_analysis_by_country_engagement', df_metricas_by_country_engajamento)
df_metricas_by_country_engajamento

,country,classified_campaign_goal,results,purchase,purchase_conversion_value,amount_spent,count,pct_results,pct_purchase,pct_purchase_conversion_value,pct_amount_spent,pct_count,full_country_name,country_population,gdp_per_capita
0,IT,Engajamento/Visualização,40304,150.0,5005.17,1600.38,52,0.01,0.15,0.15,0.14,0.01,Italy,59554023.0,39003.316095
1,ES,Engajamento/Visualização,40239,94.0,3261.22,1304.25,49,0.01,0.10,0.10,0.11,0.01,Spain,47351567.0,33509.012798
2,FR,Engajamento/Visualização,51564,97.0,3307.06,1198.94,48,0.02,0.10,0.10,0.10,0.01,France,67391582.0,44690.934540
3,DE,Engajamento/Visualização,30114,123.0,4302.58,1122.78,49,0.01,0.12,0.13,0.10,0.01,Germany,83240525.0,54343.226508
4,US,Engajamento/Visualização,22868,97.0,3358.29,969.15,37,0.01,0.10,0.10,0.08,0.01,United States,329484123.0,82769.412211
5,BE,Engajamento/Visualização,17334,88.0,3023.45,962.35,56,0.01,0.09,0.09,0.08,0.01,Belgium,11555997.0,54700.909324
6,NL,Engajamento/Visualização,18016,90.0,2984.29,941.72,49,0.01,0.09,0.09,0.08,0.01,Netherlands,16655799.0,64572.005956
7,GB,Engajamento/Visualização,11969,67.0,2144.74,492.07,33,0.0,0.07,0.06,0.04,0.01,United Kingdom,67215293.0,49463.855462
8,CH,Engajamento/Visualização,4857,48.0,1711.51,405.05,54,0.0,0.05,0.05,0.04,0.01,Switzerland,8654622.0,99564.710026
9,AT,Engajamento/Visualização,6640,33.0,1180.69,307.23,48,0.0,0.03,0.04,0.03,0.01,Austria,8917205.0,56033.573792


### Tests

In [ ]:
df_metrics_by_country_conv = df_metrics_by_country.query('classified_campaign_goal == "Conversão"')
df_metrics_by_country_eng = df_metrics_by_country.query('classified_campaign_goal == "Engajamento/Visualização"')

df_teste = (
    df_metrics_by_country.groupby("country")
    .agg({
        "results": "sum",
        "reach": "mean",
        "add_to_cart": "sum",
        "initiate_checkout": "sum",
        "purchase": "sum",
        "amount_spent": "sum",
        "purchase_conversion_value": "sum",
        "links_clicks": "sum",
    })
)

df_teste = df_teste.div(df_teste.sum()) * 100
df_teste = df_teste.reset_index()
df_teste = df_teste.sort_values(by="purchase_conversion_value", ascending=False)
df_teste.head(12)

In [83]:
country_grouped.columns

Index(['country', 'classified_campaign_goal', 'classified_result_indicator',
       'results', 'result_rate', 'reach', 'frequency', 'cpc_link',
       'cpc_general', 'cost_per_1000_people_reached', 'ctr', 'add_to_cart',
       'cost_per_add_to_cart', 'initiate_checkout',
       'cost_per_initiate_checkout', 'purchase', 'cost_per_purchase',
       'amount_spent', 'purchase_conversion_value', 'links_clicks', 'roi',
       'cart_to_checkout_conversion', 'checkout_to_purchase_conversion',
       'full_country_name', 'population_country', 'gdp_per_capita'],
      dtype='object')

In [88]:
countries = ["DE", "NL", "BE"]
country_grouped[country_grouped["country"].isin(countries)].groupby(["country"]).agg({"roi": "sum", 
                                                                                      "cart_to_checkout_conversion":"mean",
                                                                                      "checkout_to_purchase_conversion": "mean"}).sort_values(by="cart_to_checkout_conversion", ascending=False)


,roi,cart_to_checkout_conversion,checkout_to_purchase_conversion
country,,,
BE,9.297850,0.170238,0.140374
DE,7.977832,0.166907,0.158355
NL,9.703831,0.147886,0.250005
